![](images/EscUpmPolit_p.gif "UPM")

# Telco Churn

In [230]:
import matplotlib.pyplot as plt
%matplotlib inline
import random
import numpy as np
import pandas as pd
from sklearn import datasets, svm, cross_validation, tree, preprocessing, metrics
import sklearn.ensemble as ske
import seaborn as sns
sns.set(color_codes=True)
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [103]:
df = pd.read_csv('telco-churn/train.csv')
df[:5]

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1062832,Male,0,No,No,1,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Credit card (automatic),20.55,20.55,0
1,675028,Female,0,No,No,50,Yes,Yes,DSL,No,...,No,Yes,Yes,Yes,Two year,Yes,Mailed check,79.60,4024.2,0
2,11139681,Male,1,Yes,No,59,Yes,No,Fiber optic,Yes,...,Yes,No,No,No,One year,Yes,Credit card (automatic),82.95,4903.15,0
3,1522201755,Female,0,No,No,52,Yes,Yes,DSL,No,...,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),80.95,4233.95,0
4,100644,Male,0,Yes,No,34,Yes,Yes,Fiber optic,No,...,No,No,No,Yes,Month-to-month,No,Mailed check,88.85,3000.25,0


In [104]:
df.count()

customerID          5543
gender              5543
SeniorCitizen       5543
Partner             5543
Dependents          5543
tenure              5543
PhoneService        5543
MultipleLines       5543
InternetService     5543
OnlineSecurity      5543
OnlineBackup        5543
DeviceProtection    5543
TechSupport         5543
StreamingTV         5543
StreamingMovies     5543
Contract            5543
PaperlessBilling    5543
PaymentMethod       5543
MonthlyCharges      5543
TotalCharges        5543
Churn               5543
dtype: int64

Se comprueba si tenemos algún elemento nulo en el dataset.

In [105]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

Se comprueba el tipo de datos que contiene el dataset

In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5543 entries, 0 to 5542
Data columns (total 21 columns):
customerID          5543 non-null int64
gender              5543 non-null object
SeniorCitizen       5543 non-null int64
Partner             5543 non-null object
Dependents          5543 non-null object
tenure              5543 non-null int64
PhoneService        5543 non-null object
MultipleLines       5543 non-null object
InternetService     5543 non-null object
OnlineSecurity      5543 non-null object
OnlineBackup        5543 non-null object
DeviceProtection    5543 non-null object
TechSupport         5543 non-null object
StreamingTV         5543 non-null object
StreamingMovies     5543 non-null object
Contract            5543 non-null object
PaperlessBilling    5543 non-null object
PaymentMethod       5543 non-null object
MonthlyCharges      5543 non-null float64
TotalCharges        5543 non-null object
Churn               5543 non-null int64
dtypes: float64(1), int64(4), objec

Se filtran los elementos que no son de tipo numérico, es decir aquellos que no son ni int64 ni float64. Estos serán los elementos que se codificarán de manera continua.

In [107]:
df.dtypes[df.dtypes == object]

gender              object
Partner             object
Dependents          object
PhoneService        object
MultipleLines       object
InternetService     object
OnlineSecurity      object
OnlineBackup        object
DeviceProtection    object
TechSupport         object
StreamingTV         object
StreamingMovies     object
Contract            object
PaperlessBilling    object
PaymentMethod       object
TotalCharges        object
dtype: object

Vemos los valores de los elementos anteriores de tipo object.

In [87]:
print("gender: "+df['gender'].unique())
print("Parther: "+df['Partner'].unique())
print("Dependents: "+df['Dependents'].unique())
print("PhoneService: "+df['PhoneService'].unique())
print("MultipleLines: "+df['MultipleLines'].unique())
print("InternetService: "+df['InternetService'].unique())
print("OnlineSecurity: "+df['OnlineSecurity'].unique())
print("OnlineBackup: "+df['OnlineBackup'].unique())
print("DeviceProtection: "+df['DeviceProtection'].unique())
print("TechSupport: "+df['TechSupport'].unique())
print("StreamingTV: "+df['StreamingTV'].unique())
print("StreamingMovies: "+df['StreamingMovies'].unique())
print("Contract: "+df['Contract'].unique())
print("PaperlessBilling: "+df['PaperlessBilling'].unique())
print("PaymentMethod: "+df['PaymentMethod'].unique())
print("TotalCharges: "+df['TotalCharges'].unique())

['gender: Male' 'gender: Female']
['Parther: No' 'Parther: Yes']
['Dependents: No' 'Dependents: Yes']
['PhoneService: Yes' 'PhoneService: No']
['MultipleLines: No' 'MultipleLines: Yes' 'MultipleLines: No phone service']
['InternetService: No' 'InternetService: DSL'
 'InternetService: Fiber optic']
['OnlineSecurity: No internet service' 'OnlineSecurity: No'
 'OnlineSecurity: Yes']
['OnlineBackup: No internet service' 'OnlineBackup: Yes' 'OnlineBackup: No']
['DeviceProtection: No internet service' 'DeviceProtection: No'
 'DeviceProtection: Yes']
['TechSupport: No internet service' 'TechSupport: Yes' 'TechSupport: No']
['StreamingTV: No internet service' 'StreamingTV: Yes' 'StreamingTV: No']
['StreamingMovies: No internet service' 'StreamingMovies: Yes'
 'StreamingMovies: No']
['Contract: Month-to-month' 'Contract: Two year' 'Contract: One year']
['PaperlessBilling: No' 'PaperlessBilling: Yes']
['PaymentMethod: Credit card (automatic)' 'PaymentMethod: Mailed check'
 'PaymentMethod: Bank t

Lo siguiente será comprobar si existen duplicados. En este caso comprobamos que no tenemos información duplicada.

In [108]:
df.duplicated().any()

False

A continuación se pasa a codificar los valores como variables continuas.

In [110]:
df.loc[df["gender"] == "Male", "gender"] = 0
df.loc[df["gender"] == "Female", "gender"] = 1

df.loc[df["Partner"] == "Yes", "Partner"] = 1
df.loc[df["Partner"] == "No", "Partner"] = 0

df.loc[df["Dependents"] == "Yes", "Dependents"] = 1
df.loc[df["Dependents"] == "No", "Dependents"] = 0

df.loc[df["PhoneService"] == "Yes", "PhoneService"] = 1
df.loc[df["PhoneService"] == "No", "PhoneService"] = 0

df.loc[df["MultipleLines"] == "No", "MultipleLines"] = 0
df.loc[df["MultipleLines"] == "No phone service", "MultipleLines"] = 0
df.loc[df["MultipleLines"] == "Yes", "MultipleLines"] = 1

df.loc[df["InternetService"] == "No", "InternetService"] = 0
df.loc[df["InternetService"] == "DSL", "InternetService"] = 1
df.loc[df["InternetService"] == "Fiber optic", "InternetService"] = 2

df.loc[df["OnlineSecurity"] == "No", "OnlineSecurity"] = 0
df.loc[df["OnlineSecurity"] == "No internet service", "OnlineSecurity"] = 0
df.loc[df["OnlineSecurity"] == "Yes", "OnlineSecurity"] = 1

df.loc[df["OnlineBackup"] == "No", "OnlineBackup"] = 0
df.loc[df["OnlineBackup"] == "No internet service", "OnlineBackup"] = 0
df.loc[df["OnlineBackup"] == "Yes", "OnlineBackup"] = 1

df.loc[df["DeviceProtection"] == "No", "DeviceProtection"] = 0
df.loc[df["DeviceProtection"] == "No internet service", "DeviceProtection"] = 0
df.loc[df["DeviceProtection"] == "Yes", "DeviceProtection"] = 1

df.loc[df["TechSupport"] == "No", "TechSupport"] = 0
df.loc[df["TechSupport"] == "No internet service", "TechSupport"] = 0
df.loc[df["TechSupport"] == "Yes", "TechSupport"] = 1

df.loc[df["StreamingTV"] == "No", "StreamingTV"] = 0
df.loc[df["StreamingTV"] == "No internet service", "StreamingTV"] = 0
df.loc[df["StreamingTV"] == "Yes", "StreamingTV"] = 1

df.loc[df["StreamingMovies"] == "No", "StreamingMovies"] = 0
df.loc[df["StreamingMovies"] == "No internet service", "StreamingMovies"] = 0
df.loc[df["StreamingMovies"] == "Yes", "StreamingMovies"] = 1

df.loc[df["Contract"] == "Two year", "Contract"] = 0
df.loc[df["Contract"] == "One year", "Contract"] = 1
df.loc[df["Contract"] == "Month-to-month", "Contract"] = 2

df.loc[df["PaperlessBilling"] == "Yes", "PaperlessBilling"] = 1
df.loc[df["PaperlessBilling"] == "No", "PaperlessBilling"] = 0

df.loc[df["PaymentMethod"] == "Bank transfer (automatic)", "PaymentMethod"] = 0
df.loc[df["PaymentMethod"] == "Mailed check", "PaymentMethod"] = 1
df.loc[df["PaymentMethod"] == "Electronic check", "PaymentMethod"] = 2
df.loc[df["PaymentMethod"] == "Credit card (automatic)", "PaymentMethod"] = 3

df["TotalCharges"] = df["TotalCharges"].apply(pd.to_numeric, errors='coerce')

df[-5:]

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
5538,241896052,0,0,0,0,39,1,0,1,0,...,0,1,0,0,0,1,0,49.80,1971.15,0
5539,1917607,1,0,1,1,34,1,0,1,1,...,1,0,0,0,1,1,1,60.80,2042.05,0
5540,1651761,1,0,1,0,28,1,0,0,0,...,0,0,0,0,2,0,1,20.50,563.05,0
5541,8238468,0,1,1,0,69,1,1,2,0,...,1,0,0,1,2,1,0,95.35,6382.00,0
5542,5531185,0,0,0,0,63,1,1,1,1,...,1,1,0,0,1,0,0,68.80,4111.35,0


Tras realizar las modificaciones se vuelve a comprobar que no se tenga ningún elemento Null

In [111]:
df.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        10
Churn                0
dtype: int64

Se comprueba que la columna TotalCharges contiene 10 valores nulos. Se sustituyen por la media.

In [112]:
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median())

In [113]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

Se vuelve a comprobar que no haya duplicados.

In [114]:
df.duplicated().any()

False

Ahora separamos los datos en dos arrays: Uno X que contendra todos los valores de cada fila menos los de churn. y otro Y que contendrá solo los datos de churn para esa fila. El algoritmo de clasificacion comparara los valores de atributo de X a sus correspondientes valores de Y para detectar patrones de como diferentes valores de atributo afectan a la permanencia de un cliente.

In [255]:
X = df.drop(['Churn'], axis=1).values
y = df['Churn'].values

Ahora dividimos los arrays X e Y en dos partes, un set de entrenamiento y otro de testeo. Daremos el set de entrenamiento al algoritmo de clasificación para crear un modelo entrenado. Una vez el modelo se haya formado, lo utilizaremos para clasificar el set de testeo, lo que nos permitira determinar la precision del modelo. Aquí he hecho un reparto de 20-80, de tal manera que el 80% del dataset se utilizará para entrenamiento y el 20% para testeo:

In [256]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.2)

Primero comenzaremos con un arbol de decision (Tecnica aprendida en clase.) El arbol de decision examinara una variable cada vez, y se dividira entre 1 de 2 ramas dependiendo del resultado del valor obtenido. 
De tal manera que el árbol primero se divide por sexo, y después por clase, ya que habrá aprendido en la secuencia de entrenamiento que estas son las dos principales caracteristicas que determinan la suerte de un individuo. 

In [257]:
clf_dt = tree.DecisionTreeClassifier(max_depth=10)

In [258]:
clf_dt.fit (X_train, y_train)
clf_dt.score (X_test, y_test)

0.76825969341749323

Como sabemos, la precisión del modelo puede variar dependiendo de que filas sean seleccionadas para las secuencias de entrenamiento y testeo. Para que esto no condicione los resultados vamos a poner un factor de aleatoriedad:

In [259]:
shuffle_validator = cross_validation.ShuffleSplit(len(X), n_iter=20, test_size=0.2, random_state=0)
def test_classifier(clf):
    scores = cross_validation.cross_val_score(clf, X, y, cv=shuffle_validator)
    print("Accuracy: %0.4f (+/- %0.2f)" % (scores.mean(), scores.std()))

In [260]:
test_classifier(clf_dt)

Accuracy: 0.7560 (+/- 0.01)


In [261]:
y_true, y_pred = y_test, clf_dt.predict(X_test)
print(classification_report(y_true, y_pred))
print (f1_score(y_true, y_pred))

             precision    recall  f1-score   support

          0       0.85      0.83      0.84       822
          1       0.55      0.59      0.57       287

avg / total       0.77      0.77      0.77      1109

0.566610455312


In [264]:
print("Prediction ", clf_dt.predict(X))

Prediction  [0 0 0 ..., 0 0 0]


Se intentan optimizar los parámetros empleando Grid Search

In [298]:
# set of parameters to test
tuned_parameters = [{'max_depth': np.arange(3, 10),
                     'criterion': ['gini', 'entropy'], 
                     'splitter': ['best', 'random'],
                     'min_samples_leaf': [2, 5, 10],
                     'class_weight':['balanced'],
                     'max_leaf_nodes': [None, 5, 10, 20]
                    }]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    # cv = the fold of the cross-validation cv, defaulted to 5
    gs = GridSearchCV(DecisionTreeClassifier(), tuned_parameters, cv=10, scoring='%s_weighted' % score)
    gs.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(gs.best_params_)
    print()
    print("Detailed classification report:")
    print()

    y_true, y_pred = y_test, gs.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 8, 'max_leaf_nodes': 10, 'min_samples_leaf': 5, 'splitter': 'random'}

Detailed classification report:

             precision    recall  f1-score   support

          0       0.91      0.70      0.79       822
          1       0.48      0.80      0.60       287

avg / total       0.80      0.73      0.74      1109


# Tuning hyper-parameters for recall

Best parameters set found on development set:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 5, 'max_leaf_nodes': 10, 'min_samples_leaf': 2, 'splitter': 'random'}

Detailed classification report:

             precision    recall  f1-score   support

          0       0.90      0.72      0.80       822
          1       0.49      0.76      0.59       287

avg / total       0.79      0.73      0.75      1109




Se modifica el modelo usando los parámetros optimizados obtenidos previamente.

In [299]:
clf_dt = tree.DecisionTreeClassifier(class_weight='balanced', criterion='gini',max_depth=8,
                                    max_leaf_nodes=20,min_samples_leaf=2,splitter='random')

In [300]:
clf_dt.fit (X_train, y_train)
clf_dt.score (X_test, y_test)

0.75653742110009015

In [301]:
shuffle_validator = cross_validation.ShuffleSplit(len(X), n_iter=20, test_size=0.2, random_state=0)
def test_classifier(clf):
    scores = cross_validation.cross_val_score(clf, X, y, cv=shuffle_validator)
    print("Accuracy: %0.4f (+/- %0.2f)" % (scores.mean(), scores.std()))

In [302]:
test_classifier(clf_dt)

Accuracy: 0.7505 (+/- 0.01)


In [304]:
print(classification_report(y_true, y_pred))
print (f1_score(y_true, y_pred))

             precision    recall  f1-score   support

          0       0.90      0.72      0.80       822
          1       0.49      0.76      0.59       287

avg / total       0.79      0.73      0.75      1109

0.593197278912


In [281]:
dfTest = pd.read_csv('telco-churn/test_nolabel.csv')

In [282]:
dfTest.loc[dfTest["gender"] == "Male", "gender"] = 0
dfTest.loc[dfTest["gender"] == "Female", "gender"] = 1

dfTest.loc[dfTest["Partner"] == "Yes", "Partner"] = 1
dfTest.loc[dfTest["Partner"] == "No", "Partner"] = 0

dfTest.loc[dfTest["Dependents"] == "Yes", "Dependents"] = 1
dfTest.loc[dfTest["Dependents"] == "No", "Dependents"] = 0

dfTest.loc[dfTest["PhoneService"] == "Yes", "PhoneService"] = 1
dfTest.loc[dfTest["PhoneService"] == "No", "PhoneService"] = 0

dfTest.loc[dfTest["MultipleLines"] == "No", "MultipleLines"] = 0
dfTest.loc[dfTest["MultipleLines"] == "No phone service", "MultipleLines"] = 0
dfTest.loc[dfTest["MultipleLines"] == "Yes", "MultipleLines"] = 1

dfTest.loc[dfTest["InternetService"] == "No", "InternetService"] = 0
dfTest.loc[dfTest["InternetService"] == "DSL", "InternetService"] = 1
dfTest.loc[dfTest["InternetService"] == "Fiber optic", "InternetService"] = 2

dfTest.loc[dfTest["OnlineSecurity"] == "No", "OnlineSecurity"] = 0
dfTest.loc[dfTest["OnlineSecurity"] == "No internet service", "OnlineSecurity"] = 0
dfTest.loc[dfTest["OnlineSecurity"] == "Yes", "OnlineSecurity"] = 1

dfTest.loc[dfTest["OnlineBackup"] == "No", "OnlineBackup"] = 0
dfTest.loc[dfTest["OnlineBackup"] == "No internet service", "OnlineBackup"] = 0
dfTest.loc[dfTest["OnlineBackup"] == "Yes", "OnlineBackup"] = 1

dfTest.loc[dfTest["DeviceProtection"] == "No", "DeviceProtection"] = 0
dfTest.loc[dfTest["DeviceProtection"] == "No internet service", "DeviceProtection"] = 0
dfTest.loc[dfTest["DeviceProtection"] == "Yes", "DeviceProtection"] = 1

dfTest.loc[dfTest["TechSupport"] == "No", "TechSupport"] = 0
dfTest.loc[dfTest["TechSupport"] == "No internet service", "TechSupport"] = 0
dfTest.loc[dfTest["TechSupport"] == "Yes", "TechSupport"] = 1

dfTest.loc[dfTest["StreamingTV"] == "No", "StreamingTV"] = 0
dfTest.loc[dfTest["StreamingTV"] == "No internet service", "StreamingTV"] = 0
dfTest.loc[dfTest["StreamingTV"] == "Yes", "StreamingTV"] = 1

dfTest.loc[dfTest["StreamingMovies"] == "No", "StreamingMovies"] = 0
dfTest.loc[dfTest["StreamingMovies"] == "No internet service", "StreamingMovies"] = 0
dfTest.loc[dfTest["StreamingMovies"] == "Yes", "StreamingMovies"] = 1

dfTest.loc[dfTest["Contract"] == "Two year", "Contract"] = 0
dfTest.loc[dfTest["Contract"] == "One year", "Contract"] = 1
dfTest.loc[dfTest["Contract"] == "Month-to-month", "Contract"] = 2

dfTest.loc[dfTest["PaperlessBilling"] == "Yes", "PaperlessBilling"] = 1
dfTest.loc[dfTest["PaperlessBilling"] == "No", "PaperlessBilling"] = 0

dfTest.loc[dfTest["PaymentMethod"] == "Bank transfer (automatic)", "PaymentMethod"] = 0
dfTest.loc[dfTest["PaymentMethod"] == "Mailed check", "PaymentMethod"] = 1
dfTest.loc[dfTest["PaymentMethod"] == "Electronic check", "PaymentMethod"] = 2
dfTest.loc[dfTest["PaymentMethod"] == "Credit card (automatic)", "PaymentMethod"] = 3

dfTest["TotalCharges"] = dfTest["TotalCharges"].apply(pd.to_numeric, errors='coerce')
dfTest['TotalCharges'] = dfTest['TotalCharges'].fillna(dfTest['TotalCharges'].median())
dfTest[-5:]


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
1495,1014207636,1,0,1,1,2,0,0,1,0,0,0,1,0,0,2,0,0,29.05,44.75
1496,17133913,1,1,1,0,5,1,0,2,0,0,0,0,0,0,2,1,2,69.35,341.60
1497,14165210,1,0,0,0,46,1,0,0,0,0,0,0,0,0,0,1,2,21.10,937.10
1498,24777679,0,1,0,0,21,1,1,2,0,1,1,0,1,1,2,1,2,104.35,2271.85
1499,161514055,0,0,1,1,68,1,0,2,1,1,0,1,0,1,0,0,0,95.10,6683.40


In [283]:
X2 = dfTest

In [284]:
churn = clf_dt.predict(X2)

In [285]:
dataframeResult = dfTest.filter(['customerID'], axis=1)

In [286]:
dataframeResult['Churn'] = churn

In [287]:
dataframeResult[:]

,customerID,Churn
0,21135993,0
1,13130474,0
2,7533169,1
3,2114231208,0
4,3524728,1
5,192125049,0
6,172428986,0
7,21329444,0
8,232256498,0
9,162223094,1


In [288]:
dataframeResult.to_csv('out.csv', sep='\t')